In [8]:
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import normalize
from scipy.sparse import dia_matrix
from scipy.optimize import minimize, shgo
import scipy

from sklearn.model_selection import train_test_split

In [2]:
path = "../data/data.xlsx"
df = pd.read_excel(path)
df.Дата = df.Дата.astype('datetime64[ns]')

In [3]:
features = [
    ['intercept', 'Кривизна'],
    ['intercept', 'Кривизна', 'Профиль пути'],
    ['intercept', 'Кривизна', 'n_feature_1'], # Профиль пути * Макс. число вагонов в сходе
    ['intercept', 'Кривизна', 'n_feature_2'], # 1 - Макс. число вагонов в сходе / Общее количество вагонов
    ['intercept', 'Кривизна', 'n_feature_3'], # Скорость * Загрузка
    ['intercept', 'Кривизна', 'Профиль пути', 'n_feature_3'],
    ['intercept', 'Кривизна', 'Профиль пути', 'n_feature_3', 'n_feature_2'],
    ['intercept', 'Профиль пути', 'n_feature_3', 'n_feature_2']
]

target = ['Количество сшедших вагонов']

In [103]:
lambdas = [
    (lambda t: np.exp(t)),
    (lambda t: np.exp(-t * t)),
    (lambda t: 1 + np.sqrt(np.abs(25 - (t-5)*(t-5)))),
    (lambda t: (t-1)*(t-1)),
    (lambda t: 1 / (1 + t*t)),
    (lambda t: t * (np.pi / 2.0 - np.arctan(-t)) + 1),
    (lambda t: np.log(1 + t*t) + 1),
]

log_lambdas = [
    (lambda t: t),
    (lambda t: -t * t),
    (lambda t: np.log(1 + np.sqrt(np.abs(25 - (t-5)*(t-5))))),
    (lambda t: np.log((t-1)*(t-1))),
    (lambda t: -np.log(1 + t*t)),
    (lambda t: np.log(t * (np.pi / 2.0 - np.arctan(-t)) + 1)),
    (lambda t: np.log(np.log(1 + t*t) + 1)),
]

In [77]:
class PoissonRegression():
    theta = None
    lambda_index = 0
    X = None
    y = None
    bounds = [(-1000, 1000), (-1000, 1000), (-1000, 1000), (-1000, 1000), (-1000, 1000)]
    new_bounds = [(None, None), (None, None), (None, None), (None, None), (None, None)]
    
    def __init__(self, df, features, target):
        self.df = df
        self.features = features
        self.target = target
        self.theta = np.zeros(len(self.features[0]))
    
    def log_likelihood_function_compact(self, theta):
        result = 0
        for i in range(len(self.y)):
            dot_prod = float(np.dot(self.X[i], theta))
            lambda_value = lambdas[self.lambda_index](dot_prod)
            log_lambda_value = log_lambdas[self.lambda_index](dot_prod)
            #print(lambda_value)
            result += (self.y[i][0] - 1) * log_lambda_value - lambda_value
        return -result
    
    def get_log_likelihood_const(self):
        result = 0
        for i in range(len(self.y)):
            result += -np.log(scipy.special.factorial(self.y[i]))
        return result
    
    def construct_plot(self):
        plt.yticks(range(1, 30,2))
        plt.plot(data['Количество сшедших вагонов'], color='black', marker='^', linestyle='none', markersize=5)
        #plt.plot(FIXME, color='red', marker='+', linestyle='none', markersize=5)
        plt.xlabel('номер наблюдения')
        plt.ylabel('кол-во сшедших вагонов')
        plt.legend(['source' ,'lambda_1'], loc=5)
        plt.show()
    
    def fit_all(self):
        for feature in self.features:
            data = self.df[feature + self.target].dropna()
            self.y = data[self.target]
            self.y = np.array(self.y)
            self.X = data.drop(self.target, axis=1)
            self.X = np.matrix(self.X)
            print(feature)
            for lambda_index in range(len(lambdas)):
                #print("lambda_index", lambda_index)
                self.lambda_index = lambda_index
                
                results = dict()
                results['shgo'] = shgo(self.log_likelihood_function_compact, self.bounds[:len(feature)])
                #print(results['shgo'])
                print(results['shgo']['fun'] + self.get_log_likelihood_const())
                print(results['shgo']['success'])
                print(results['shgo']['x'])
                print()
                #self.construct_plot()
                
                #min_llfc = minimize(self.log_likelihood_function_compact, np.zeros(len(feature)))
            print("########################################################")

In [104]:
pRegressor = PoissonRegression(df, features, target)
pRegressor.fit_all()

['intercept', 'Кривизна']
[-265.05635587]
True
[   1.17663977 -271.62551142]

[-214.66218274]
True
[-2.73694797e-11  0.00000000e+00]

[-214.66218274]
True
[0. 0.]

[-265.18492478]
True
[ -0.79854568 208.15177928]

[-214.66218274]
True
[-2.73694797e-11  0.00000000e+00]

[-265.04783487]
True
[   0.94835424 -236.2893394 ]

[-264.6663515]
True
[   2.66181393 -549.7289914 ]

########################################################
['intercept', 'Кривизна', 'Профиль пути']

<ipython-input-103-3655a5e8d18a>:2: RuntimeWarning: overflow encountered in exp
  (lambda t: np.exp(t)),



[-275.18553909]
True
[   1.17290318 -160.85035451   39.2961505 ]

[-217.87042328]
True
[-2.73694797e-11  0.00000000e+00  0.00000000e+00]

[-278.52868572]
True
[   0.55043225 -112.98959909   35.68112338]

[-278.0145109]
True
[ -0.83371375 212.69897542 -56.9265697 ]

[-217.87042328]
True
[-2.73694797e-11  0.00000000e+00  0.00000000e+00]

[-279.55923183]
True
[   1.05191327 -281.74568162   83.44281457]

[-282.51837017]
True
[   3.28364711 -486.79224254  276.35808383]

########################################################
['intercept', 'Кривизна', 'n_feature_1']
[-288.09981464]
True
[   1.29025355 -201.69345478    0.81851829]

[-221.87042328]
True
[-2.73694797e-11  0.00000000e+00  0.00000000e+00]



<ipython-input-103-3655a5e8d18a>:2: RuntimeWarning: overflow encountered in exp
  (lambda t: np.exp(t)),
<ipython-input-103-3655a5e8d18a>:2: RuntimeWarning: overflow encountered in exp
  (lambda t: np.exp(t)),
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


[-292.13593315]
True
[  0.84287729 -77.65647708   1.76096833]

[-291.62146599]
True
[ -0.93871636 280.753527    -1.43886108]

[-221.87042328]
True
[-2.73694797e-11  0.00000000e+00  0.00000000e+00]

[-294.54634425]
True
[   1.21245882 -426.4098919     2.4939739 ]

[-296.62787756]
True
[    3.90693742 -1000.             7.26268052]

########################################################
['intercept', 'Кривизна', 'n_feature_2']
[-299.4135261]
True
[   2.11226982 -180.42532241   -2.35720266]

[-218.66218274]
True
[-2.73694797e-11  0.00000000e+00  0.00000000e+00]



...

[-244.83031673]
True
[ 9.29352269  3.40284729  2.57567522 -0.11394161]

[-282.90419428]
True
[ 2.93386487e-01  2.44791202e+02 -5.24675953e+01 -2.44099775e-02]

[-209.0685958]
True
[-7.07090063e-14  0.00000000e+00  0.00000000e+00 -2.00601451e-10]

[-283.88111441]
True
[-4.66087855e-01 -3.28739403e+02  7.58828149e+01  3.27852746e-02]

[-278.48325762]
True
[-3.13685004e-01 -6.33810800e+02  7.19187274e+00  8.24899080e-02]

########################################################
['intercept', 'Кривизна', 'Профиль пути', 'n_feature_3', 'n_feature_2']


<ipython-input-103-3655a5e8d18a>:2: RuntimeWarning: overflow encountered in exp
  (lambda t: np.exp(t)),


[-298.10457509]
True
[ 9.20629830e-01 -1.05353670e+02  3.43367668e+01  2.12149030e-02
 -2.23823032e+00]

[-211.0685958]
True
[-6.99064063e-14  0.00000000e+00  0.00000000e+00 -2.01144033e-10
  0.00000000e+00]

[-211.0685958]
True
[0. 0. 0. 0. 0.]

[-297.40626425]
True
[-9.82505685e-01  2.42257779e+02 -2.26209593e+01 -1.29605807e-02
  1.73184593e+00]

[-211.0685958]
True
[-6.99656641e-14  0.00000000e+00  0.00000000e+00 -2.01314537e-10
  0.00000000e+00]

[-297.50842915]
True
[ 1.28025114e+00 -3.73537510e+02  4.11037890e+01  1.68902002e-02
 -2.34402031e+00]

[-293.37510009]
True
[ 2.68871388e+00 -5.79479083e+01  2.35448009e+02  7.88651473e-02
 -8.15421055e+00]

########################################################
['intercept', 'Профиль пути', 'n_feature_3', 'n_feature_2']
[-291.43713615]
True
[ 5.51508839e-01  3.92803481e+01  2.54177582e-02 -2.13831419e+00]

[-209.0685958]
True
[-7.07737609e-14  0.00000000e+00 -2.01492897e-10  0.00000000e+00]

[-209.0685958]
True
[0. 0. 0. 0.]

[-286.9

<ipython-input-103-3655a5e8d18a>:2: RuntimeWarning: overflow encountered in exp
  (lambda t: np.exp(t)),


[-286.94994797]
True
[ 0.31226162 26.22986496  0.02698084 -1.96985321]

[-287.20124141]
True
[ 1.90841693e+00  2.81505717e+02  7.62294180e-02 -6.89975165e+00]

########################################################


In [108]:
np.e ** -214

1.150749706249289e-93